In [1]:
import pandas as pd
import numpy as np

In [2]:
# kohort barn
b0, b1 = 72, 77

# kohort foreldre
f0, f1 = 42, 54

# ige/rank
ige = True

In [3]:
df = pd.read_csv('kpi.csv', header=0, sep=';')
kpi = df[df.columns[1]]
kpi.index = df[df.columns[0]]
kpi = kpi[kpi.index <= 2015]
kpi = kpi.str.replace(',','.')
kpi = kpi.astype(float)
kpi = 100 / kpi
kpi.head()

Unnamed: 0
2015    1.000000
2014    1.021450
2013    1.042753
2012    1.064963
2011    1.071811
Name: Årsgj.snitt, dtype: float64

In [4]:
with open('test.txt', 'w') as f:
    f.write('create-dataset barn \n')
    f.write('import BEFOLKNING_FOEDSELS_AAR_MND as fodmnd \n')
    f.write('generate kohort = int( fodmnd / 100 ) \n')
    f.write(f'drop if kohort < 19{b0} | kohort > 19{b1} \n')

    for år in range(2000, 2015+1):
        f.write('import INNTEKT_PGIVINNT '+str(år)+'-01-01 as i'+str(år)[2:]+'\n')
        f.write('generate i'+str(år)[2:]+'ij = i'+str(år)[2:]+'*'+f'{kpi[år]:.4f}\n')

    inntekter = [f'innt{år}' for år in range(28,38+1)]

    for inntår in inntekter:
        f.write(f'generate {inntår} = 0 \n')
    
    for kohort in range(b0, b1+1):
        for år in range(28,38+1):
            f.write(f'replace innt{år} = i'+str(kohort+år)[1:]+'ij if kohort == 19'+str(kohort)+'\n')
    for år in range(29,37+1):
        f.write(f'generate inntekt_barn_{år} = (innt{år-1} + innt{år} + innt{år+1}) / 3 \n' )
    string = 'generate inntekt_barn_3438 = ('
    for år in range(34,38):
        string += f'innt{år} +'
    string += 'innt38)/5 \n'
    f.write(string)
    f.write('import BEFOLKNING_FAR_FNR as far_fnr \n')

#####  Fedre

In [5]:
with open('test.txt', 'a+') as f:
    f.write('create-dataset far \n')
    f.write('import BEFOLKNING_FOEDSELS_AAR_MND as fodmnd \n')
    f.write('generate kohortf = int( fodmnd / 100 ) \n')
    f.write('drop if kohortf > 19'+str(f1)+' \n')
    f.write('drop if kohortf < 19'+str(f0)+' \n')
    
    for år in range(1982, 1999+1):
        f.write('import INNTEKT_PGIVINNT '+str(år)+'-01-01 as if'+str(år)[2:]+'\n')
        f.write('generate if'+str(år)[2:]+'ij = if'+str(år)[2:]+'*'+f'{kpi[år]:.4f}\n')
    for år in range(2000, 2011+1):
        f.write('import INNTEKT_PGIVINNT '+str(år)+'-01-01 as if'+str(år)[2:]+'\n')
        f.write('generate if'+str(år)[2:]+'ij = if'+str(år)[2:]+'*'+f'{kpi[år]:.4f}\n')
        
    inntekter = [f'innt{år}' for år in range(40,57+1)]
    
    for inntår in inntekter:
        f.write(f'generate {inntår} = 0 \n')

    for kohort in range(f0, f1+1):
        for år in range(40,56+1):
            if kohort+år < 100:
                f.write(f'replace innt{år} = if'+str(kohort+år)+'ij if kohortf == 19'+str(kohort)+'\n')
            else:
                f.write(f'replace innt{år} = if'+str(kohort+år)[1:]+'ij if kohortf == 19'+str(kohort)+'\n')
    
    
    string_merge = "merge "
    for år in range(42,55+1,2):
        f.write(f'generate inntekt_far_{år} = (innt{år-2} + innt{år-1} + innt{år} + innt{år+1}+ innt{år+2}) / 5 \n' )
        string_merge += f'inntekt_far_{år} '
    
    
    for år in range(41,50+1,2):
        string = f'generate inntekt_far_40{år} = ('
        for j in range(40,år):
            string += f'innt{j} + '
        string += f'innt{år})/{år-39} \n'
        f.write(string)
        string_merge += f'inntekt_far_40{år} '
        
        
    string_merge += "into barn on far_fnr \n"
    f.write(string_merge)

##### nasjonal

In [6]:
if ige:
    with open('test.txt', 'a+') as f:
        f.write('use barn \n')

        f.write('generate log_inntekt_barn_29 = ln(inntekt_barn_29) \n')
        f.write('generate log_inntekt_barn_31 = ln(inntekt_barn_31) \n')
        f.write('generate log_inntekt_barn_33 = ln(inntekt_barn_33) \n')
        f.write('generate log_inntekt_barn_35 = ln(inntekt_barn_35) \n')
        f.write('generate log_inntekt_barn_37 = ln(inntekt_barn_37) \n')
        f.write('generate log_inntekt_barn_3438 = ln(inntekt_barn_3438) \n')
        f.write('generate log_inntekt_far_42 = ln(inntekt_far_42) \n')
        f.write('generate log_inntekt_far_44 = ln(inntekt_far_44) \n')
        f.write('generate log_inntekt_far_46 = ln(inntekt_far_46) \n')
        f.write('generate log_inntekt_far_48 = ln(inntekt_far_48) \n')
        f.write('generate log_inntekt_far_50 = ln(inntekt_far_50) \n')
        f.write('generate log_inntekt_far_52 = ln(inntekt_far_52) \n')
        f.write('generate log_inntekt_far_54 = ln(inntekt_far_54) \n')
        f.write('generate log_inntekt_far_4041 = ln(inntekt_far_4041) \n')
        f.write('generate log_inntekt_far_4043 = ln(inntekt_far_4043) \n')
        f.write('generate log_inntekt_far_4045 = ln(inntekt_far_4045) \n')
        f.write('generate log_inntekt_far_4047 = ln(inntekt_far_4047) \n')
        f.write('generate log_inntekt_far_4049 = ln(inntekt_far_4049) \n')

        f.write('regress log_inntekt_barn_29 log_inntekt_far_4049 \n')
        f.write('regress log_inntekt_barn_31 log_inntekt_far_4049 \n')
        f.write('regress log_inntekt_barn_33 log_inntekt_far_4049 \n')
        f.write('regress log_inntekt_barn_35 log_inntekt_far_4049 \n')
        f.write('regress log_inntekt_barn_37 log_inntekt_far_4049 \n')

        f.write('regress log_inntekt_barn_3438 log_inntekt_far_42 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_44 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_46 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_48 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_50 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_52 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_54 \n')

        f.write('regress log_inntekt_barn_3438 log_inntekt_far_4041 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_4043 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_4045 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_4047 \n')
        f.write('regress log_inntekt_barn_3438 log_inntekt_far_4049 \n')

In [7]:
if not ige:
    with open('test.txt', 'a+') as f:
        f.write('use barn \n')
        f.write('generate prosentil_barn_29 = quantile(inntekt_barn_29,100) \n')
        f.write('generate prosentil_barn_31 = quantile(inntekt_barn_31,100) \n')
        f.write('generate prosentil_barn_33 = quantile(inntekt_barn_33,100) \n')
        f.write('generate prosentil_barn_35 = quantile(inntekt_barn_35,100) \n')
        f.write('generate prosentil_barn_37 = quantile(inntekt_barn_37,100) \n')
        f.write('generate prosentil_barn_3438 = quantile(inntekt_barn_3438,100) \n')
        f.write('generate prosentil_far_42 = quantile(inntekt_far_42,100) \n')
        f.write('generate prosentil_far_44 = quantile(inntekt_far_44,100) \n')
        f.write('generate prosentil_far_46 = quantile(inntekt_far_46,100) \n')
        f.write('generate prosentil_far_48 = quantile(inntekt_far_48,100) \n')
        f.write('generate prosentil_far_50 = quantile(inntekt_far_50,100) \n')
        f.write('generate prosentil_far_52 = quantile(inntekt_far_52,100) \n')
        f.write('generate prosentil_far_54 = quantile(inntekt_far_54,100) \n')
        f.write('generate prosentil_far_4041 = quantile(inntekt_far_4041,100) \n')
        f.write('generate prosentil_far_4043 = quantile(inntekt_far_4043,100) \n')
        f.write('generate prosentil_far_4045 = quantile(inntekt_far_4045,100) \n')
        f.write('generate prosentil_far_4047 = quantile(inntekt_far_4047,100) \n')
        f.write('generate prosentil_far_4049 = quantile(inntekt_far_4049,100) \n')

        f.write('regress prosentil_barn_29 prosentil_far_4049 \n')
        f.write('regress prosentil_barn_31 prosentil_far_4049 \n')
        f.write('regress prosentil_barn_33 prosentil_far_4049 \n')
        f.write('regress prosentil_barn_35 prosentil_far_4049 \n')
        f.write('regress prosentil_barn_37 prosentil_far_4049 \n')

        f.write('regress prosentil_barn_3438 prosentil_far_42 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_44 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_46 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_48 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_50 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_52 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_54 \n')

        f.write('regress prosentil_barn_3438 prosentil_far_4041 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_4043 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_4045 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_4047 \n')
        f.write('regress prosentil_barn_3438 prosentil_far_4049 \n')